# Installations and imports

Instalaciones necesarias

In [13]:
!pip install fasttext
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296185 sha256=c946b6b5e826e422cea84ff3162fa04a8c23cdb334304eac004cb92aac6b23ab
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00


Imports necesarios

In [14]:
import fasttext
import torch
import json
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Mounted at /content/drive


# Dataset cleaning and preparations


Datasets en portugues

In [18]:
# Cargamos los dataset de portugues que vamos a utilizar
pt_legal = load_dataset("rufimelo/PortugueseLegalSentences-v2")
pt_wiki = load_dataset("jvanz/portuguese_wikipedia_sentences")

README.md:   0%|          | 0.00/474 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/62.6M [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/62.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

evaluation-00000-of-00001.parquet:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63387 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10565 [00:00<?, ? examples/s]

Generating evaluation split:   0%|          | 0/10565 [00:00<?, ? examples/s]

In [19]:
# Modificamos los dataframes para obtener la estructura deseada
train_df_legal = pd.DataFrame(pt_legal['train'])
train_df_legal['split'] = 'train'
train_df_legal = train_df_legal.iloc[:, 1:]

test_df_legal = pd.DataFrame(pt_legal['test'])
test_df_legal['split'] = 'test'
test_df_legal = test_df_legal.iloc[:, 1:]

val_df_legal = pd.DataFrame(pt_legal['validation'])
val_df_legal['split'] = 'val'
val_df_legal = val_df_legal.iloc[:, 1:]

legal_df = pd.concat([train_df_legal, test_df_legal, val_df_legal], ignore_index=True)
legal_df['language'] = 'pt'

In [20]:
train_df_wiki = pd.DataFrame(pt_wiki['train'])
train_df_wiki['split'] = 'train'

test_df_wiki = pd.DataFrame(pt_wiki['test'])
test_df_wiki['split'] = 'test'

val_df_wiki = pd.DataFrame(pt_wiki['evaluation'])
val_df_wiki['split'] = 'val'

wiki_df = pd.concat([train_df_wiki, test_df_wiki, val_df_wiki], ignore_index=True)
wiki_df['language'] = 'pt'

In [21]:
# Juntamos los datasets en uno solo
pt_df = pd.concat([legal_df, wiki_df], ignore_index=True)

# Guardamos el dataset en un CSV
pt_df.to_csv('pt_dataset.csv', index=False)

print(f'Esta es la estructura final del dataset en portugués que tiene un total de {len(pt_df)} frases: ')
pt_df.head()

Esta es la estructura final del dataset en portugués que tiene un total de 552025 frases: 


,text,split,language
0,Deve a Ré ser condenada a pagar ao Autor a dif...,train,pt
1,"O valor equivalente a 9/12 de uma remuneração,...",train,pt
2,"O Acórdão recorrido está, salvo o devido respe...",train,pt
3,"Há, por isso, erro de julgamento quando o Trib...",train,pt
4,Ora é pacificamente aceite que na tarefa de qu...,train,pt


Datasets en gallego

In [23]:
# TODO: Path para Iker
# ruta_press = '/content/drive/My Drive/4.Curso/NLP/Proyecto/press_and_blogs.jsonl'

# TODO: Path para Luis
ruta_press = '/content/drive/My Drive/4º Carrera/NLP/Proyecto/press_and_blogs.jsonl'

In [24]:
# Cargar el contenido del corpus en gallego del archivo JSON
press_gl = []

with open(ruta_press, 'r') as archivo:
    for linea in archivo:
        press_gl.append(json.loads(linea))

In [25]:
# Modificamos el dataframe para conseguir nuestra estructura deseada
press_df = pd.DataFrame(press_gl)
press_df = press_df.iloc[:, 1:2]

train_df_press, temp_df = train_test_split(press_df, test_size=0.3, random_state=42)  # 70% train, 30% temp
val_df_press, test_df_press = train_test_split(temp_df, test_size=0.5, random_state=42)  # 15% val, 15% test

train_df_press['split'] = 'train'
test_df_press['split'] = 'test'
val_df_press['split'] = 'val'

press_df = pd.concat([train_df_press, test_df_press, val_df_press], ignore_index=True)
press_df['language'] = 'gl'

# Guardamos el dataset en un CSV
press_df.to_csv('gl_dataset.csv', index=False)

print(f'Esta es la estructura final del dataset en gallego que tiene un total de {len(press_gl)} frases: ')
press_df.head()

Esta es la estructura final del dataset en gallego que tiene un total de 598375 frases: 


,text,split,language
0,Pequeno homenaxe a mellor película de cine bél...,train,gl
1,Wilhelm Bunsen (1811-99). Químico e físico ale...,train,gl
2,O Parlamento receita bicicletas para combater ...,train,gl
3,As asociacións estudiantís apoian a folga\nAs ...,train,gl
4,"O venres visitáronnos os de Son a Aguiño, vimo...",train,gl


# Embeddings no contextuales (fasttext)

Embedding fasttext gallego

In [28]:
# Embedding fasttext gallego cbow de 300 dimensiones
# TODO: Path para Iker
# gl_embedding = fasttext.load_model('/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/gl_300.bin')

# TODO: Path para Luis
# gl_fasttext = fasttext.load_model('/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/gl_300.bin')

Embedding fasttext portugués

In [29]:
# Embedding fasttext portugués cbow de 300 dimensiones
# TODO: Path para Iker
# pt_fasttext = fasttext.load_model('/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/pt_300.bin')

# TODO: Path para Luis
# pt_embedding = fasttext.load_model('/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/pt_300.bin')

#Embeddings contextuales (BERT)

Bertinho (BERT gallego)

In [ ]:
# Cargamos bertinho y el tokenizer para el dataset gallego
gl_tokenizer = AutoTokenizer.from_pretrained("dvilares/bertinho-gl-base-cased")
bertinho = AutoModelForMaskedLM.from_pretrained("dvilares/bertinho-gl-base-cased")

# Texto en gallego
texto = "Ola, este é un exemplo en galego."

# Tokenización del texto
inputs = gl_tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Generar embeddings
with torch.no_grad():
    outputs = bertinho(**inputs)

sentence_embedding = outputs.logits
print(sentence_embedding)

BERTimbau (BERT portugués)

In [ ]:
# Cargamos BERTimbau y el tokenizer para el dataset portugues
pt_tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
bertimbau = AutoModelForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Texto en portugués
texto = "Olá, este é um exemplo em português."

# Tokenización del texto
inputs = pt_tokenizer(texto, return_tensors="pt", padding=True, truncation=True)

# Generar embeddings
with torch.no_grad():
    outputs = bertimbau(**inputs)

sentence_embedding = outputs.logits

print(sentence_embedding)

# Usar los embeddings

In [ ]:
# Función para obtener el embedding de un texto en un modelo específico
def obtener_embedding_contextual(texto, tokenizer, model):
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.logits
    return embedding

def obtener_embedding_no_contextual(texto, model):
    tokens = word_tokenize(texto)
    embeddings_palabras = [model.get_word_vector(token) for token in tokens]

    # No sabemos si es necesario hacer la media de la oración
    embedding_texto = np.mean(embeddings_palabras, axis=0)
    return embedding_texto

In [ ]:
# Función para generar y concatenar embeddings de gallego y portugués
def generar_embeddings_concatenados(texto, contextual):
    if contextual:
      emb_gallego = obtener_embedding_contextual(texto, gl_tokenizer, bertinho)
      emb_portugues = obtener_embedding_contextual(texto, pt_tokenizer, bertimbau)
    else:
      emb_gallego = obtener_embedding_no_contextual(texto, gl_fasttext)
      emb_portugues = obtener_embedding_no_contextual(texto, pt_fasttext)

    embedding_concatenado = np.concatenate([emb_gallego, emb_portugues])
    return embedding_concatenado

In [ ]:
# Ejemplo de como se generarían embeddings concatenados para el dataset de entrenamiento
train_data = pt_df[pt_df['split'] == 'train']
train_embeddings = [generar_embeddings_concatenados(texto) for texto in train_data['text']]